In [39]:
import json
import jieba
from src.FastTextRank4Word import FastTextRank4Word
from src.FastTextRank4Sentence import FastTextRank4Sentence
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from langchain_community.embeddings import GPT4AllEmbeddings
from neo4j import GraphDatabase

In [2]:
stopwords = set()
with open('dict/stopwords.txt', 'r', encoding="utf8") as file:
    stopwords = file.readlines()
    stopwords = [stopword.strip('\n').strip() for stopword in stopwords]
except_file = open("dict/hippo_exception_word.txt", encoding='utf-8')
exception = except_file.read().split(',')
exception.append(" ")

In [3]:
def normalize_document(text, stopwords, exception):
    punct = set(u''':!),.:;?]}$¢'"、。〉》」』】〕〗〞︰︱︳﹐､﹒﹔﹕﹖﹗﹚﹜﹞！），．：；？｜｝︴︶︸︺︼︾﹀﹂﹄﹏､～￠々‖•·ˇˉ―--′’”([{£¥'"‵〈《「『【〔〖（［｛￡￥〝︵︷︹︻︽︿﹁﹃﹙﹛﹝（｛“‘-—_…''')
    punct |= set(exception)
    words = [word for word in jieba.cut(text, cut_all=True) if (len(word.strip()) >= 2) and (word not in stopwords) and ( not any(ext in word for ext in punct) )]
    return " ".join(words)

In [4]:
jieba.initialize()
jieba.set_dictionary('dict/dict.big.txt')
jieba.load_userdict('dict/mydic.txt')
stops = ()
with open('dict/stopwords.txt', 'r', encoding='utf8') as f:
    stops = f.read().split('\n') 

Building prefix dict from c:\Users\sean.chang\yfy\creative_corner\Supervisor_llm\dict\dict.big.txt ...
Loading model from cache C:\Users\SEAN~1.CHA\AppData\Local\Temp\jieba.u5cd32b1f7ee506be41af6ed3342ccc1e.cache


Loading model cost 0.831 seconds.
Prefix dict has been built successfully.


In [5]:
# query
query = "被糾正機關是行政院原住民族委員會及花蓮縣政府"

In [6]:
query_extraction_model = FastTextRank4Word(tol=0.0001, window=5)
query_extraction = [query_extraction_model.summarize(query, 5)][0]
query_extraction

['花蓮縣政府', '原住民族委員會', '行政院']

-  Why do I set jieba_load_userdict works when using FastTextRank4Word??

In [31]:
with open('organized_final_result.txt', 'r', encoding='utf-8') as text_file:
    data = json.load(text_file)

In [34]:
data_kw = eval(data['34829.pdf']['metadata']['kw'])
target = data['34829.pdf']['target']
knowledge_graph = data['34829.pdf']['knowledge_graph']

In [33]:
data_kw

['政府', '行政院', '案件', '原住民', '計畫', '漁探機', '本院', '行政效率', '部落', '未能']

In [38]:
knowledge_graph

[{'entity1': '立法委員', 'relationship': '函轉', 'entity2': '民眾陳訴'},
 {'entity1': '行政院原住民族委員會', 'relationship': '補助', 'entity2': '花蓮區漁會'},
 {'entity1': '花蓮區漁會',
  'relationship': '採購',
  'entity2': '「三機一體彩色漁探衛星導航電子海圖儀及船用ＤＳＢ無線電對講機」'},
 {'entity1': '行政院原住民族委員會', 'relationship': '監督', 'entity2': '花蓮區漁會'},
 {'entity1': '花蓮縣政府', 'relationship': '監督', 'entity2': '花蓮區漁會'}]

In [21]:
embedding_model = "gpt4all-falcon-newbpe-q4_0.gguf"
embedder = GPT4AllEmbeddings(embedding_model=embedding_model)
embedded_query = embedder.embed_query(query)
embedded_target = embedder.embed_query(target)
embedded_kw = embedder.embed_documents(data_kw)

In [15]:
# query_target similarity
cosine_similarity([embedded_target, embedded_query])[0][1]

0.9753698810140833

In [42]:
# query_kw similairy
def kw_similarity(embedded_query, embedded_kw, kw_threshold):
    count = 0
    # kw_query_list=  []
    for i in embedded_kw:
        sim_value = cosine_similarity([i, embedded_query])[0][1]
        # kw_query_list.append(sim_value)
        if sim_value > kw_threshold:
            count += 1

    similarity = count / len(embedded_kw)
    return similarity

In [43]:
similarity= kw_similarity(embedded_query, embedded_kw, kw_threshold=0.7)
similarity

0.2

In [86]:
URI = "bolt://localhost:7688"
user = 'neo4j'
password = "!QAZ2wsx"
AUTH = (user, password)

with GraphDatabase.driver(URI, auth=AUTH, encrypted=False) as driver:
    driver.verify_connectivity()

In [49]:
def create_relationships(tx, data):
    for item in data:
        tx.run("MERGE (a:Entity {name: $entity1}) "
               "MERGE (b:Entity {name: $entity2}) "
               "MERGE (a)-[{type: $relationship}]->(b)",
               entity1=item["entity1"], relationship=item["relationship"], entity2=item["entity2"])

In [127]:
class Neo4jHandler:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def insert_relationship(self, entity1, relationship, entity2):
        with self.driver.session() as session:
            session.write_transaction(self._create_and_return_relationship, entity1, relationship, entity2)

    @staticmethod
    def _create_and_return_relationship(tx, entity1, relationship, entity2):
        query = (
            "MERGE (a:Entity {name: $entity1}) "
            "MERGE (b:Entity {name: $entity2}) "
             f"MERGE (a)-[r:{relationship}]->(b) "
            "RETURN a, b, r"
        )
        tx.run(query, entity1=entity1, relationship=relationship, entity2=entity2)

    @staticmethod
    def _fetch_node_rela(tx):
        query = """
        MATCH (n:Entity)-[r]->(m:Entity)
        RETURN n.name AS source, type(r) AS relationship, m.name AS target
        """
        result = tx.run(query)
        for record in result:
            print(f"{record['source']} {record['relationship']} {record['target']}")

    def fetch_all_data(self):
        with GraphDatabase.driver(uri=URI, auth=("neo4j", "neo4j")) as driver:
            with driver.session() as session:
                session.read_transaction(self._fetch_node_rela)

        

In [128]:
handler = Neo4jHandler(URI, user, password)

In [76]:
for sublist in data['34829.pdf']['knowledge_graph']:
    print(sublist)
    # for item in sublist:

{'entity1': '立法委員', 'relationship': '函轉', 'entity2': '民眾陳訴'}
{'entity1': '行政院原住民族委員會', 'relationship': '補助', 'entity2': '花蓮區漁會'}
{'entity1': '花蓮區漁會', 'relationship': '採購', 'entity2': '「三機一體彩色漁探衛星導航電子海圖儀及船用ＤＳＢ無線電對講機」'}
{'entity1': '行政院原住民族委員會', 'relationship': '監督', 'entity2': '花蓮區漁會'}
{'entity1': '花蓮縣政府', 'relationship': '監督', 'entity2': '花蓮區漁會'}


In [125]:
for key, value in data.items():
    filename = key
    knowledge_graph = value['knowledge_graph']
    for item in knowledge_graph:
        handler.insert_relationship(item['entity1'], item['relationship'], item['entity2'])

C:\Users\sean.chang\AppData\Local\Temp\ipykernel_26068\3821333987.py:11: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._create_and_return_relationship, entity1, relationship, entity2)


In [129]:
handler.fetch_all_data()


立法委員 函轉 民眾陳訴
行政院原住民族委員會 補助 花蓮區漁會
花蓮區漁會 採購 「三機一體彩色漁探衛星導航電子海圖儀及船用ＤＳＢ無線電對講機」
行政院原住民族委員會 監督 花蓮區漁會
花蓮縣政府 監督 花蓮區漁會
立法委員 制定 法律
研究主持人 接受 政府委託研究計畫
衛生署 補助 國立台灣師範大學
衛生署人員 出國 國外
公教人員 申請 出國審核
環保署 規定 主管機關
主管機關 查核 清除機構
清除機構 勾稽比對 主管機關
環保署 提出書面資料 本院
各級主管機關 針對 清除機構申報之營運紀錄
台北榮總 誤解規定 相關法令
台北榮總 提升資格 廠商投標資格
台北榮總 延長工期 停車場工程
台北榮總 未遵照工期 評輝營造公司
評輝營造公司 建議工期 七二〇天
退輔會 通函規定 各榮家
受訪單位 擇定 SBR-CASS
行政院環境保護署 函覆 退輔會
退輔會 發包 桃園榮家
桃園榮家 辦理 污水處理廠工程
教育部 辦理 二○○一年國際生物奧林匹亞競賽參賽計畫
教育部 委託 台灣師範大學
曾哲明 指示 陳仙姿
陳仙姿 篡改 陳宗佑的初試成績
陳家楨 要求 曾哲明
苗栗縣卓蘭鎮公所 排除招標規定 卓蘭鎮老庄區域排水震災修復工程
苗栗縣卓蘭鎮公所 未核定工期展延 卓蘭鎮老庄區域排水震災修復工程
苗栗縣政府 未確實監辦 卓蘭鎮老庄區域排水震災修復工程
苗栗縣卓蘭鎮公所 施工品質不良 卓蘭鎮老庄區域排水震災修復工程
苗栗縣卓蘭鎮公所 採限制性招標 卓蘭鎮老庄區域排水震災修復工程
台鐵局 違反 政府採購法第三十七條第一項
台鐵局機務處台北機廠 違反 政府採購法第四十八條第二項
台鐵局 違反 政府採購公告及公報發行辦法第六條
台鐵局餐旅服務總所 疏漏 決標結果公告應登載決標金額
台鐵局 違反 政府採購法第一百零一條至第一百零三條


C:\Users\sean.chang\AppData\Local\Temp\ipykernel_26068\1007173019.py:36: DeprecationWarning: read_transaction has been renamed to execute_read
  session.read_transaction(self._fetch_node_rela)


In [121]:
def fetch_node_rela(tx):
    query = """
    MATCH (n:Entity)-[r]->(m:Entity)
    RETURN n.name AS source, type(r) AS relationship, m.name AS target
    """
    result = tx.run(query)
    for record in result:
        print(f"{record['source']} {record['relationship']} {record['target']}")


In [122]:
def fetch_all_data():
    with GraphDatabase.driver(uri=URI, auth=("neo4j", "neo4j")) as driver:
        with driver.session() as session:
            session.read_transaction(fetch_node_rela)
fetch_all_data()

立法委員 函轉 民眾陳訴
行政院原住民族委員會 補助 花蓮區漁會
花蓮區漁會 採購 「三機一體彩色漁探衛星導航電子海圖儀及船用ＤＳＢ無線電對講機」
行政院原住民族委員會 監督 花蓮區漁會
花蓮縣政府 監督 花蓮區漁會
立法委員 制定 法律
研究主持人 接受 政府委託研究計畫
衛生署 補助 國立台灣師範大學
衛生署人員 出國 國外
公教人員 申請 出國審核
環保署 規定 主管機關
主管機關 查核 清除機構
清除機構 勾稽比對 主管機關
環保署 提出書面資料 本院
各級主管機關 針對 清除機構申報之營運紀錄
台北榮總 誤解規定 相關法令
台北榮總 提升資格 廠商投標資格
台北榮總 延長工期 停車場工程
台北榮總 未遵照工期 評輝營造公司
評輝營造公司 建議工期 七二〇天
退輔會 通函規定 各榮家
受訪單位 擇定 SBR-CASS
行政院環境保護署 函覆 退輔會
退輔會 發包 桃園榮家
桃園榮家 辦理 污水處理廠工程
教育部 辦理 二○○一年國際生物奧林匹亞競賽參賽計畫
教育部 委託 台灣師範大學
曾哲明 指示 陳仙姿
陳仙姿 篡改 陳宗佑的初試成績
陳家楨 要求 曾哲明
苗栗縣卓蘭鎮公所 排除招標規定 卓蘭鎮老庄區域排水震災修復工程
苗栗縣卓蘭鎮公所 未核定工期展延 卓蘭鎮老庄區域排水震災修復工程
苗栗縣政府 未確實監辦 卓蘭鎮老庄區域排水震災修復工程
苗栗縣卓蘭鎮公所 施工品質不良 卓蘭鎮老庄區域排水震災修復工程
苗栗縣卓蘭鎮公所 採限制性招標 卓蘭鎮老庄區域排水震災修復工程
台鐵局 違反 政府採購法第三十七條第一項
台鐵局機務處台北機廠 違反 政府採購法第四十八條第二項
台鐵局 違反 政府採購公告及公報發行辦法第六條
台鐵局餐旅服務總所 疏漏 決標結果公告應登載決標金額
台鐵局 違反 政府採購法第一百零一條至第一百零三條


C:\Users\sean.chang\AppData\Local\Temp\ipykernel_26068\3570948381.py:4: DeprecationWarning: read_transaction has been renamed to execute_read
  session.read_transaction(fetch_node_rela)
